### Import Libs

In [1]:
import pandas as pd 
import os 
import constant as c
import sys
import math
import numpy as np
from datetime import datetime
sys.path.append('../')
from util.get_coord_vw import GeoCoderVworld
import warnings
warnings.filterwarnings("ignore")

In [2]:
# We get all raw data for make data view
ssd_path = c.SSD_PATH
address_path = os.path.join(ssd_path, 'address_202304061732.csv')
title_path = os.path.join(ssd_path, 'title_202304061735.csv')
jijigu_path = os.path.join(ssd_path, 'jijigu_202304061738.csv')
building_path = os.path.join(ssd_path, 'building_202304061737.csv')
pnubjdcd_path = os.path.join(ssd_path, 'pnubjdcd.txt')
address = pd.read_csv(address_path, encoding = 'cp949')
title = pd.read_csv(title_path, encoding = 'cp949')
jijigu = pd.read_csv(jijigu_path, encoding = 'cp949')
building = pd.read_csv(building_path, encoding = 'cp949')
pnubjdcd = pd.read_csv(pnubjdcd_path, encoding = 'cp949', sep =  '\t')
pnubjdcd = pnubjdcd[pnubjdcd['법정동명'].str.contains('서울특별시')]
pnubjdcd = pnubjdcd[pnubjdcd['폐지여부'] == '존재']

df_bd = pd.merge(address, building[['pnu', 'bdnm']], on = 'pnu')
df_bd = pd.merge(df_bd, \
               title[['pnu', 'mainpurpscd', 'mainpurpscdnm', 'etcpurps', 'platarea', \
                'archarea', 'bcrat', 'totarea', 'vlrat', 'heit', 'grndflrcnt', 'ugrndflrcnt', \
                    'rideuseelvtcnt', 'emgenuseelvtcnt', 'useaprday']], on = 'pnu')
df_bd = pd.merge(df_bd, \
                jijigu[(jijigu['jijigugbcdnm'] == '용도지역코드')&(jijigu['repryn'] == 1)][['pnu', 'jijigucdnm']], on = 'pnu') # 용도지역코드만 따오고 대표여부가 참(=1)인 데이터만
# df_bd = df_bd[df_bd['sigungunm'].isin(c.CBD_NMS)]
# df_bd = df_bd[df_bd['heit'] > 10] # building taller than 10m
df_bd = df_bd.drop_duplicates(subset=['pnu'], keep='first')
# df_bd = df_bd[df_bd['mainpurpscd'].isin(c.MAINPURPSCD_IN_INTEREST)]
df_bd = df_bd.sort_values(by=['pnu'], ascending=False)
df_bd

,pnu,rlgnm,rlgcd,sigungunm,lawdongnm,linm,bun,ji,naroadnm,naroadcd,...,bcrat,totarea,vlrat,heit,grndflrcnt,ugrndflrcnt,rideuseelvtcnt,emgenuseelvtcnt,useaprday,jijigucdnm
15287259,1174011000107270000,서울특별시,11,강동구,강일동,NaN,727,0,고덕로,117403124001,...,49.98,4638.67,249.88,24.60,5,2,1,0,20130201.0,준주거지역
15287560,1174011000107230000,서울특별시,11,강동구,강일동,NaN,723,0,고덕로,117403124001,...,59.98,5207.04,296.25,28.65,5,3,2,0,20211102.0,준주거지역
15287260,1174011000107170000,서울특별시,11,강동구,강일동,NaN,717,0,고덕로,117403124001,...,0.00,311.91,0.00,3.90,1,0,0,0,20110114.0,제2종일반주거지역
15287563,1174011000107120000,서울특별시,11,강동구,강일동,NaN,712,0,상일로,117403124004,...,49.46,1083.36,62.77,15.60,2,1,0,0,20090701.0,제3종일반주거지역
15287564,1174011000107110000,서울특별시,11,강동구,강일동,NaN,711,0,상일로,117403124004,...,82.21,1397.59,182.33,11.40,3,0,0,0,20160330.0,도시지역
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47217,1111010100100040015,서울특별시,11,종로구,청운동,NaN,4,15,자하문로36길,111104100289,...,42.90,352.50,72.24,0.00,2,1,0,0,19671220.0,NaN
47218,1111010100100040014,서울특별시,11,종로구,청운동,NaN,4,14,자하문로36길,111104100289,...,29.94,813.96,87.25,11.95,3,1,0,0,20040108.0,일반주거지역
47216,1111010100100040007,서울특별시,11,종로구,청운동,NaN,4,7,자하문로36길,111104100289,...,29.79,2283.68,87.41,11.49,3,2,0,0,20000929.0,NaN
47215,1111010100100040003,서울특별시,11,종로구,청운동,NaN,4,3,자하문로36길,111104100289,...,29.69,1536.11,84.11,11.65,3,2,1,0,20070619.0,제1종일반주거지역


In [3]:
print(len(df_bd))
useaprday = [int(value) if not math.isnan(value) else 9999999 for value in df_bd['useaprday']]
useaprday = [str(value).zfill(8) for value in useaprday]
print(len(useaprday))
df_bd['useaprday'] = useaprday

349478
349478


In [4]:
bd_tval = pd.read_excel('../asset/rawfile/bdtval.xlsx')
bd_tval = bd_tval.drop('etcprc', axis=1)
bd_tval['platarea'] = bd_tval['platarea_py'] * 3.3
bd_tval['totarea'] = bd_tval['bdarea_py'] * 3.3
bd_tval['useaprday'] = pd.to_datetime(bd_tval['approvyear'], format='%y.%m.%d', errors='coerce').dt.strftime('%Y%m%d')
bd_tval['useaprday'] = bd_tval['useaprday'].fillna(bd_tval['useaprday'].str[:-2] + '01')
bd_tval = bd_tval.dropna()

In [5]:
new_rows = []
for i in range(len(bd_tval)):
# for i in range(20):
    row_data = bd_tval.iloc[i]
    filtered = df_bd[df_bd['platplc'].str.startswith(row_data.platplc_short)]
    # print("1. juso: ",len(filtered))
    
    totarea_float = float(row_data.totarea)
    filtered = filtered[(filtered['totarea'].between(totarea_float * 0.9, totarea_float * 1.1))]
    # print("2. totarea: ",len(filtered))

    platarea_float = float(row_data.platarea)
    filtered = filtered[(filtered['platarea'].between(platarea_float * 0.9, platarea_float * 1.1))]

    if len(filtered) == 1:
        new_row = row_data.values.tolist() + filtered.values.tolist()[0]
    else:
        useapryear = row_data.useaprday[:6]
        filtered = filtered[filtered['useaprday'].str.startswith(useapryear)]
    
        if len(filtered) == 1:
            new_row = row_data.values.tolist() + filtered.values.tolist()[0]
    
    if len(filtered) == 1:
        new_rows.append(new_row)
    else:
        pass

In [6]:
column_names = list(bd_tval.columns) + list(df_bd.columns)
df_bd_tval = pd.DataFrame(new_rows, columns = column_names)

In [7]:
print(len(bd_tval))
print(len(df_bd_tval))

3266
2068


In [8]:
columns_to_drop = ['platplc_short', 'jijigu', 'approvyear', 'platarea', 'totarea', 'useaprday']
df_bd_tval = df_bd_tval.drop(columns=columns_to_drop)
df_bd_tval = df_bd_tval.rename(columns={'platarea.1': 'platarea','totarea.1': 'totarea', 'useaprday.1': 'useaprday'})
df_bd_tval = df_bd_tval.drop_duplicates()
df_bd_tval = pd.merge(df_bd_tval, address[['pnu', 'x', 'y']], on='pnu', suffixes=('_left', ''), how='left')

In [37]:
df_bd_tval.to_csv("../asset/verification/df_bd_tval.csv")

In [38]:
# Read df_bd
df_bd_tval = pd.read_csv("../asset/verification/df_bd_tval.csv", index_col = 0)
df_bd_tval.sort_values(by = 'totalprc', ascending = False).bdnm[:20]

0                      힐튼호텔
1     그랜드센트럴(GRAND CENTRAL)
2                    두산타워빌딩
3                     가양이마트
4                  신한투자증권타워
5                      종로타워
6                  구두수선대114
7                 삼성생명 대치타워
8                  SK남산그린빌딩
9                 에이플러스에셋타워
10                    메트로타워
11                    강남피타워
12                  코레이트 타워
13                      NaN
14                     오투타워
15     서울로타워(Seoullo Tower)
16                    문정프라자
17                신한L Tower
18                     린스퀘어
19                  현대차증권빌딩
Name: bdnm, dtype: object